In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import locker.analysis  as ana
import datajoint as dj



Get all trials with contrast 20%, significant locking to beat or stimulus and |df|>30 to avoid confusion of stimulus and EOD.


In [ ]:
dat = ana.Decoding() * ana.Cells() * ana.Decoding.Beat() * ana.Decoding.Stimulus() * ana.Runs() \
      & dict(am=0) & 'contrast >= 10 and ABS(delta_f) > 30 and cell_type="p-unit"' #   & ['vs_stimulus >= crit_stimulus', 'vs_beat >= crit_beat'] 
dat

In [ ]:
summary = dj.U('cell_id', 'df', 'contrast').aggr(
            dat.proj('contrast', 'vs_beat', df='ROUND(delta_f, 0)'), locking='AVG(vs_beat)', trials='count(*)')
n = dj.U('cell_id', 'contrast').aggr(summary, n='count(*)')

df = pd.DataFrame((summary*n & 'n>=10 and trials>=10').fetch())
len(df)

In [ ]:
df

In [ ]:
sns.set_context('notebook')
g = sns.FacetGrid(df, col='cell_id', hue='contrast', col_wrap=4)

def plot_tuning(df, locking, **kwargs):
    df, locking = map(np.array, [df, locking])
    idx = np.argsort(df)
    plt.plot(df[idx], locking[idx], **kwargs)

g.map(plot_tuning, 'df', 'locking')
g.set_titles('{col_name}')
g.add_legend()
g.set_ylabels('locking to beat')
g.set_xlabels(r'$\Delta f$')
sns.despine(trim=True)
g.fig.set_dpi(100)
g.fig.savefig('delta_f_vs_beat.png')